In [ ]:
import gc
import json
import os
import shutil

In [ ]:
import utils.gemini_functions as gemini_functions
import utils.functions as functions
gemini_functions.configurar_api()

In [ ]:
# Paso 1: Leer el archivo JSON de entrada
json_data = gemini_functions.read_json_file('scraped_texts.json')

In [ ]:
# Limpiar el JSON eliminando las claves con el valor "Error al scrapear"
json_limpio = functions.limpiar_json(json_data, "Error al scrapear")

total_claves = len(json_limpio.keys())  # Contar el número total de claves
print(f"Total de claves en el JSON: {total_claves}")

In [ ]:
# Directorios
carpeta_jsons = "jsons"
carpeta_respuestas = "respuestas"
carpeta_respuestas_json = os.path.join(carpeta_respuestas, "respuestas_json")

# Crear carpetas si no existen
os.makedirs(carpeta_jsons, exist_ok=True)
os.makedirs(carpeta_respuestas, exist_ok=True)
os.makedirs(carpeta_respuestas_json, exist_ok=True)

In [ ]:
# Fragmentar el JSON
fragmentos = functions.fragmentar_json(json_limpio, claves_por_fragmento=20)

# Guardar los fragmentos en la carpeta 'jsons'
functions.guardar_fragmentos(fragmentos, carpeta_destino=carpeta_jsons)

In [ ]:
# Obtener los archivos de fragmentos en jsons/
archivos_fragmentos = sorted(
    [f for f in os.listdir(carpeta_jsons) if f.endswith(".json")]
)

In [ ]:
# Cargar el prompt desde un archivo
archivo_prompt = "prompts/analisis_de_conflicto.txt"
prompt = gemini_functions.cargar_prompt(archivo_prompt)

In [ ]:
# Procesar cada fragmento de JSON y enviara Gemini IA
for i, archivo_json in enumerate(archivos_fragmentos, start=1):
    ruta_json = os.path.join(carpeta_jsons, archivo_json)

     # Cargar el fragmento desde el archivo
    with open(ruta_json, "r", encoding="utf-8") as file:
        fragmento = json.load(file)

    # Llamada a la función para enviar la solicitud a Gemini con el fragmento actual
    response = gemini_functions.enviar_a_gemini(prompt, fragmento)

    # Procesar la respuesta
    if response:
        # print(f"Respuesta con JSON para conflicto {i}:", response)

        #imprimir la primera clave del json
        primera_clave = list(fragmento.keys())[0]

        # Guardar la respuesta en un archivo de texto con nombre secuencial
        archivo_salida = f"respuesta_gemini_{i}.txt"
        gemini_functions.guardar_respuesta_en_txt(response, archivo_salida)
    else:
        print(f"No se obtuvo una respuesta válida para conflicto {i}.")

In [ ]:
# Obtener los archivos de respuestas de Gemini
archivos_respuestas = sorted(
    [f for f in os.listdir(carpeta_respuestas) if f.endswith(".txt")]
)

# Extraer JSON de cada archivo de respuesta y guardarlo en respuestas_json/
for archivo_txt in archivos_respuestas:
    ruta_txt = os.path.join(carpeta_respuestas, archivo_txt)
    json_extraido = functions.extraer_json_de_txt(ruta_txt)

    if json_extraido:
        # Filtrar los fragmentos de JSON que contienen "descripcion_conflicto" con el texto no relevante
        json_limpio = {}
        for clave, valor in json_extraido.items():
            # Verificar si "descripcion_conflicto" contiene la descripción no relevante
            if isinstance(valor, dict) and "descripcion_conflicto" in valor:
                descripcion_conflicto = valor["descripcion_conflicto"]
                if "No se describe un conflicto social." in descripcion_conflicto:
                    # Si contiene el texto no relevante, omitimos esta clave del JSON
                    continue
            # Si no se eliminó la clave, la mantenemos en el JSON limpio
            json_limpio[clave] = valor

        if json_limpio:
            archivo_json = archivo_txt.replace(".txt", ".json")
            ruta_json = os.path.join(carpeta_respuestas_json, archivo_json)

            # Guardar el JSON limpio en un archivo
            with open(ruta_json, "w", encoding="utf-8") as file:
                json.dump(json_limpio, file, indent=4, ensure_ascii=False)

            print(f"JSON extraído de {archivo_txt} y guardado en {ruta_json}")
        else:
            print(f"El JSON de {archivo_txt} está vacío después de la limpieza.")
    else:
        print(f"No se pudo extraer JSON válido de {archivo_txt}")

In [ ]:
# Unir los JSON en un solo archivo

# Directorio de respuestas JSON y archivo de salida
carpeta_respuestas_json = "respuestas/respuestas_json"
archivo_json_salida = "join_respuestas_gemini.json"

# Unir todos los JSON en un solo archivo
functions.unir_jsons(carpeta_respuestas_json, archivo_json_salida)

#eliminar la carpeta respuestas
shutil.rmtree(carpeta_respuestas)
shutil.rmtree(carpeta_jsons)